# Interroger des bases de données SQL avec Python 

Dans ce brief, nous allons explorer toutes les étapes nécessaires pour connecter Python et SQL:

- Importation de la bibliothèque
- Connexion au serveur et création de la base de données
- Créer des tables
- Remplissage des tables
- Lecture des données
- Mise à jour des enregistrements
- Suppression d'enregistrements
- Création d'enregistrements à partir de listes

## 1. Exigences et connexion au serveur


#### Créer un environnement Python qu'on nommera "PySQL"

In [ ]:
"""
Anaconda Navigator > Environments > Create > Name : PySQL | version : 3.8.16
"""

#### Installer les bibliothèque mysql.connector et Pandas dans l'environnement

In [ ]:
"""
conda install -c anaconda mysql-connector-python
"""

In [ ]:
"""
conda install -c anaconda pandas
"""

In [ ]:
"""
conda install -c anaconda ipykernel
"""

#### Créer une connexion au serveur SQL 

Ensuite, nous voulons définir une fonction dans Python qui se connecte à notre serveur SQL. Pour ce faire, nous utilisons la méthode mysql.connector.connect().


In [ ]:
import mysql.connector

def connect_serv (host, user, pswd):
    server = mysql.connector.connect(
    host = host,
    user = user,
    password = pswd
    )
    return(server)

connect_serv("192.168.20.118", "grogu", "grogu")

---

## 2. Création d'une BDD

#### Créer une BDD

Maintenant nous définissons une fonction pour créer une nouvelle base de données sur notre serveur. Ici, nous utilisons cursor.execute() pour exécuter une commande SQL CREATE DATABASE.

In [ ]:
def create_db (server, name):
    cursor = server.cursor()
    cursor.execute("CREATE DATABASE " + name)

create_db(connect_serv("192.168.20.118", "grogu", "grogu"), "pysql")

#### Modifier la fonction de connexion au serveur, créer la fonction de connexion à la base de données

Maintenant que nous avons créé une BD, modifions notre fonction pour créer une nouvelle fonction permettant de se connecter directement à cette BD. Cela s'avérera plus utile que la simple connexion à notre serveur.

In [ ]:
def connect_db (host, user, pswd, db_name):
    db = mysql.connector.connect(
    host = host,
    database = db_name,
    user = user,
    password = pswd)
    return(db)

connect_db("192.168.20.118", "grogu", "grogu", "pysql")

#### Créer une fonction d'exécution de requête


La dernière étape de cette phase consiste à créer une fonction qui nous permettra d'exécuter des requêtes écrites en SQL. Cela va être extrêmement utile !

In [ ]:
def exe(server, request):
    cursor = server.cursor()
    cursor.execute(request)
    server.commit()

---

## 3. Création de Tables

#### Créer une table "Enseignant"

Maintenant, créons notre première table à l'intérieur de notre BD, en utilisant nos fonctions nouvellement définies

In [ ]:
# Assigner notre commande SQL à une variable python en utilisant des guillemets triples pour créer une chaîne de caractères de plusieurs lignes.

request = """
CREATE TABLE `pysql`.`Teacher`(
    `teacher_id` INT NOT NULL AUTO_INCREMENT ,
    `first_name` VARCHAR(40) NOT NULL ,
    `last_name` VARCHAR(40) NOT NULL ,
    `dob` DATE NOT NULL ,
    `phone_no` VARCHAR(20) NOT NULL ,
    `language_1` VARCHAR(3) NOT NULL ,
    `language_2` VARCHAR(3) ,
    `tax_id` INT NOT NULL ,
    PRIMARY KEY (`teacher_id`)
)ENGINE = InnoDB;
"""

# Connexion à la base de données

db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

# Exécuter notre requête définie

exe(db, request)


#### Créer le reste des  tables de données "Client", "Participant", "Course", de la même façon

In [ ]:
#client_table
requests = []
requests.append("""
CREATE TABLE `pysql`.`Client` (
    `client_id` INT NOT NULL AUTO_INCREMENT ,
    `client_name` VARCHAR(40) NOT NULL ,
    `address` VARCHAR(60) NOT NULL , 
    `industry` VARCHAR(20) NOT NULL , 
    PRIMARY KEY (`client_id`)
)ENGINE = InnoDB;
"""
)

#Participant_table

requests.append("""
CREATE TABLE `pysql`.`Participant` (
    `participant_id` INT NOT NULL AUTO_INCREMENT , 
    `first_name` VARCHAR(20) NOT NULL , 
    `last_name` VARCHAR(20) NOT NULL , 
    `phone_no` VARCHAR(20) NOT NULL ,
    `client` INT NOT NULL , 
    PRIMARY KEY (`participant_id`)
)ENGINE = InnoDB;
"""
)

#course_table

requests.append("""
CREATE TABLE `pysql`.`Course` (
    `course_id` INT NOT NULL AUTO_INCREMENT , 
    `course_name` VARCHAR(40) NOT NULL , 
    `course_length_weeks` INT NOT NULL , 
    `language` VARCHAR(3) NOT NULL , 
    `level` VARCHAR(2) NOT NULL , 
    `start_date` DATE NOT NULL , 
    `in_school` BOOLEAN NOT NULL , 
    `teacher` INT NOT NULL , 
    `client` INT NOT NULL , 
    PRIMARY KEY (`course_id`)
)ENGINE = InnoDB;
"""
)

# Connexion à la base de données

db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

# Exécuter nos requêtes définies

for request in requests:
    exe(db, request)

#### Définir les relations entre "Client", "Participant", "Course" en se référant à l'architecture de la BDD

Maintenant, nous modifions les tables pour créer des relations de type clé étrangère et nous créons notre table finale

In [ ]:
requests = []

# Participant

requests.append("""
ALTER TABLE `Participant` 
ADD FOREIGN KEY (`client`) REFERENCES `Client` (`client_id`);
"""
)

# Course

requests.append("""
ALTER TABLE `Course`
ADD FOREIGN KEY (`client`) REFERENCES `Client` (`client_id`) ,
ADD FOREIGN KEY (`teacher`) REFERENCES `Teacher` (`teacher_id`);
"""
)

# Takes_course

requests.append("""
CREATE TABLE `Takes_Course`(
    `participant_id` INT NOT NULL,
    `course_id` INT NOT NULL,
	FOREIGN KEY (`participant_id`) REFERENCES `Participant` (`participant_id`),
    FOREIGN KEY (`course_id`) REFERENCES `Course` (`course_id`)
)ENGINE=InnoDB;
"""
)
# Connexion à la base de données

db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

# Exécuter nos requêtes définies

for request in requests:
    exe(db, request)

---

##  4. Remplissage les tables avec les données 

Remplir la table 'Teacher' avec les données du fichier Excel

Ici, nous assignons à nouveau une chaîne de plusieurs lignes avec notre commande SQL à une variable, puis nous appelons nos fonctions de création et d'execution déjà définies

In [ ]:
# Installation openpyxl
"""
conda install -c anaconda openpyxl
"""

In [ ]:
import pandas as pd

# lecture du fichier avec pandas
df = pd.read_excel('Tables.xlsx', sheet_name = "ILS DB", usecols = 'B:I', skiprows = 2, nrows = 6)

# Enumeration des valeurs (nested liste), génération des requetes pour chaque ligne
requests = []
for line in df.values.tolist():
    requests.append(f"""
    INSERT INTO `pysql`.`Teacher` VALUES (
        '{line[0]}', 
        '{line[1]}',
        '{line[2]}', 
        '{line[5]}', 
        '{line[7]}', 
        '{line[3]}', 
        '{line[4]}', 
        '{line[6]}');
    """)

# Connexion à la base de données

db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

# Exécuter nos requêtes définies

for request in requests:
    exe(db, request)

#### Vérifiez que les tables sont bien remplies 

Nous allons remplir les autres tables de la même manière

In [ ]:
# Client

df = pd.read_excel('Tables.xlsx', sheet_name = "ILS DB", usecols = 'B:E', skiprows = 11, nrows = 5)

requests = []
for line in df.values.tolist():
    requests.append(f"""
    INSERT INTO `pysql`.`Client` VALUES (
        '{line[0]}', 
        '{line[1]}',
        '{line[2]}', 
        '{line[3]}');
    """)
    
# Connexion à la base de données

db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

# Exécuter nos requêtes définies

for request in requests:
    exe(db, request)

In [ ]:
# Participant

df = pd.read_excel('Tables.xlsx', sheet_name = "ILS DB", usecols = 'B:F', skiprows = 19, nrows = 14)

requests = []
for line in df.values.tolist():
    requests.append(f"""
    INSERT INTO `pysql`.`Participant` VALUES (
        '{line[0]}', 
        '{line[1]}',
        '{line[2]}', 
        '{line[3]}',
        '{line[4]}');
    """)
    
# Connexion à la base de données

db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

# Exécuter nos requêtes définies

for request in requests:
    exe(db, request)

In [ ]:
# Course
df = pd.read_excel('Tables.xlsx', sheet_name = "ILS DB", usecols = 'B:J', skiprows = 36, nrows = 9)

requests = []
for line in df.values.tolist():
    requests.append(f"""
    INSERT INTO `pysql`.`Course` VALUES (
        '{line[0]}', 
        '{line[1]}',
        '{line[4]}', 
        '{line[2]}',
        '{line[3]}',
        '{line[5]}',
        '{1 if line[6] else 0}',
        '{line[7]}',
        '{line[8]}');
    """)
    
# Connexion à la base de données

db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

# Exécuter nos requêtes définies

for request in requests:
    exe(db, request)

In [ ]:
# Takes_Course

df = pd.read_excel('Tables.xlsx', sheet_name = "ILS DB", usecols = 'B:C', skiprows = 48, nrows = 17)
requests = []
for line in df.values.tolist():
    requests.append(f"""
    INSERT INTO `pysql`.`Takes_Course` VALUES (
        '{line[0]}', 
        '{line[1]}');
    """)
    
# Connexion à la base de données

db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

# Exécuter nos requêtes définies

for request in requests:
    exe(db, request)

---

## 5. Lire des données

Maintenant que nous avons rempli nos tables, il est temps de commencer à créer des requêtes de lecture. Pour ce faire, nous avons besoin d'une nouvelle fonction.

#### Fonction de lecture des données

In [ ]:
def read_exe(server, request, col_name = False):
    cursor = db.cursor(buffered = True)
    cursor.execute(request)
    server.commit()
    result = cursor.fetchall()
    if col_name:
        result.insert(0, cursor.column_names)
    return(result)

#### Appliquez la fonction de lecture des données sur les tables de la BDD

Nous pouvons maintenant affecter les résultats à une liste, pour les utiliser ensuite dans nos applications ou scripts python.


#### Mettez  les données dans une liste de tuples 

In [ ]:
import mysql.connector

db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

data = read_exe(db, "SELECT * FROM Client;")

print(data)

#### Mettez  les données dans un dataframe Pandas


In [ ]:
import pandas as pd

def table_to_df(db, request):
    liste = [list(line) for line in read_exe(db, request, True)]

    df = pd.DataFrame(data = liste[1:], columns = liste[0])
    
    return(df)

db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

table_to_df(db, "SELECT * FROM Course;")


---

## 6. Mise à jour des enregistrements

L'école a été informée que la Big Business Federation (id =101) avait déménagé et qu'elle se trouvait désormais au 323 Fingiertweg, 14534 Berlin3. Modifiez cette adresse dans la base de données en  utilisant la fonction 'update' 

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

exe(db, "UPDATE Client SET address = '323 Fingiertweg, 14534 Berlin3' WHERE client_id = 101;")

Vérifiez si l'adresse a été bien changée

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

table_to_df(db, "SELECT * FROM Client WHERE client_id = 101;")

---

## 7. Suppression d'enregistrements

Supprimez le cours dont l'identifiant est 20. Pour cela, utilisez la commande SQL DELETE.

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

queries = ("DELETE FROM Takes_Course WHERE course_id = 20;", "DELETE FROM Course WHERE course_id = 20;")

for query in queries:
    exe(db, query)

Vérifiez si le cours a été bien supprimé

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

table_to_df(db, "SELECT * FROM Course;")

Rétablir le cours supprimé  et vérifié qu'il est bien rétabli 

In [ ]:
requests = []

df = pd.read_excel('Tables.xlsx', sheet_name = "ILS DB", usecols = 'B:J', skiprows = 44, nrows = 1)

for line in df.values.tolist() :
    requests.append(f"""
    INSERT INTO `pysql`.`Course` VALUES (
        '{line[0]}', 
        '{line[1]}',
        '{line[4]}', 
        '{line[2]}',
        '{line[3]}',
        '{line[5]}',
        '{1 if line[6] else 0}',
        '{line[7]}',
        '{line[8]}');
    """)
    
df = pd.read_excel('Tables.xlsx', sheet_name = "ILS DB", usecols = 'B:C', skiprows = 60, nrows = 1)


for line in df.values.tolist() :
    requests.append(f"""
    INSERT INTO `pysql`.`Takes_Course` VALUES (
        '{line[0]}', 
        '{line[1]}');
    """)

# Connexion à la base de données

db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

# Exécuter nos requêtes définies

for request in requests:
    exe(db, request)

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

table_to_df(db, "SELECT * FROM Course;")

---

## 8. Création d'enregistrements à partir de listes

Ajoutons quelques nouveaux professeurs à notre table 'teacher'.
Tout d'abord, modifions notre fonction execute_query pour utiliser executemany() et accepter un argument supplémentaire.

In [ ]:
def exe_many(server, request, data):
    cursor = server.cursor()
    cursor.executemany(request, data)
    server.commit()

Maintenant, créons une liste contenant les données des nouveaux enseignants (chacune étant stockée dans un tuple), et la commande SQL pour effectuer notre action.

Notez que la commande SQL nécessite un caractère générique '%s' pour chacune des colonnes sur lesquelles nous souhaitons agir, donc dans ce cas, nous avons besoin de 8 pour les 8 colonnes pour lesquelles nous souhaitons ajouter des valeurs.

In [ ]:
new_teachers = [
    (None, 'Tom-Lou', 'Pierron', '2001-06-01', '+251210451', 'FRA', None, 54861),
    (None, 'Ethan', 'Vuillemin', '2002-07-04', '+545105455', 'FRA', 'ENG', 66514)
]

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "pysql")

exe_many(db, "INSERT INTO Teacher VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", new_teachers)

---

## CONCLUSION

Nous avons couvert beaucoup de terrain dans ces deux premières briefs. De l'utilisation de Python et du connecteur MySQL à la création d'une toute nouvelle base de données dans le serveur MariaDB, en passant par la création de tables, la définition de leurs relations mutuelles et leur alimentation en données. Nous avons vu comment créer, lire, mettre à jour et supprimer des données dans notre base de données.

Nous avons vu comment extraire des données de bases de données existantes et les charger dans des DataFrames pandas, prêtes à être analysées et travaillées en tirant parti de toutes les possibilités offertes par la pile PyData. Dans l'autre sens, nous avons également appris à prendre les données générées par nos scripts et applications Python et à les écrire dans une base de données où elles peuvent être stockées en toute sécurité pour être récupérées ultérieurement.
